In [1]:
### load libraries
import time
import ee
import os
#import numpy
import pandas
import numpy as np
#import feather
ee.Authenticate()
ee.Initialize()


Successfully saved authorization token.


In [2]:
def AddFmask(image):
    qa = image.select('pixel_qa')
    water = qa.bitwiseAnd(1 << 7)
    cloud = qa.bitwiseAnd(1 << 3)
    snow = qa.bitwiseAnd(1 << 5)
    cloudshadow = qa.bitwiseAnd(1 << 4)
    
    fmask = (water.gt(0).rename(['fmask'])
    .where(snow.gt(0), ee.Image(3))
    .where(cloudshadow.gt(0), ee.Image(2))
    .where(cloud.gt(0), ee.Image(4))
    .updateMask(qa.gte(0)))
    ## mask the fmask so that it has the same footprint as the quality (BQA) band
    return image.addBands(fmask)
      

In [3]:
def Mndwi (image):
  return(image
  .expression('(Green-Swir1) / (Swir1 + Green)', {
    'Swir1': image.select(['Swir1']),
    'Green': image.select(['Green'])
  })).rename('mndwi')

def Mbsrv(image) :
  return(image
  .expression('Green + Red', {
    'Green': image.select(['Green']),
    'Red': image.select(['Red'])
  })).rename('mbsrv')

def Mbsrn (image):
  return(image
  .expression('Nir + Swir1', {
    'Nir': image.select(['Nir']),
    'Swir1': image.select(['Swir1'])
  })).rename('mbsrn')

def Ndvi(image):
  return(image
  .expression('(Nir - Red) / (Nir + Red)', {
    'Nir': image.select(['Nir']),
    'Red': image.select(['Red'])
  })).rename('ndvi')

def Awesh(image):
  return (image.addBands(Mbsrn(image))
  .expression('Blue + 2.5 * Green + (-1.5) * mbsrn + (-0.25) * Swir2', {
    'Blue': image.select(['Blue']),
    'Green': image.select(['Green']),
    'mbsrn': Mbsrn(image).select(['mbsrn']),
    'Swir2': image.select(['Swir2'])
    }))

In [4]:
## The DSWE Function itself    
def Dswe(i):
  mndwi = Mndwi(i)
  mbsrv = Mbsrv(i)
  mbsrn = Mbsrn(i)
  awesh = Awesh(i)
  swir1 = i.select(['Swir1'])
  nir = i.select(['Nir'])
  ndvi = Ndvi(i)
  blue = i.select(['Blue'])
  swir2 = i.select(['Swir2'])
  t1 = mndwi.gt(0.124)
  t2 = mbsrv.gt(mbsrn)
  t3 = awesh.gt(0)
  t4 = (mndwi.gt(-0.44)
  .And(swir1.lt(0.900))
  .And(nir.lt(0.1500))
  .And(ndvi.lt(0.7)))
  t5 = (mndwi.gt(-0.5)
  .And(blue.lt(0.1000))
  .And(swir1.lt(0.3000))
  .And(swir2.lt(0.1000))
  .And(nir.lt(0.2500)))
  
  t = t1.add(t2.multiply(10)).add(t3.multiply(100)).add(t4.multiply(1000)).add(t5.multiply(10000))
  
  noWater = (t.eq(0)
  .Or(t.eq(1))
  .Or(t.eq(10))
  .Or(t.eq(100))
  .Or(t.eq(1000)))
  hWater = (t.eq(1111)
  .Or(t.eq(10111))
  .Or(t.eq(11011))
  .Or(t.eq(11101))
  .Or(t.eq(11110))
  .Or(t.eq(11111)))
  mWater = (t.eq(111)
  .Or(t.eq(1011))
  .Or(t.eq(1101))
  .Or(t.eq(1110))
  .Or(t.eq(10011))
  .Or(t.eq(10101))
  .Or(t.eq(10110))
  .Or(t.eq(11001))
  .Or(t.eq(11010))
  .Or(t.eq(11100)))
  pWetland = t.eq(11000)
  lWater = (t.eq(11)
  .Or(t.eq(101))
  .Or(t.eq(110))
  .Or(t.eq(1001))
  .Or(t.eq(1010))
  .Or(t.eq(1100))
  .Or(t.eq(10000))
  .Or(t.eq(10001))
  .Or(t.eq(10010))
  .Or(t.eq(10100)))
  
  iDswe = (noWater.multiply(0)
  .add(hWater.multiply(1))
  .add(mWater.multiply(2))
  .add(pWetland.multiply(3))
  .add(lWater.multiply(4)))
  
  return iDswe.rename('dswe')

In [5]:
def removeGeo(i):
    return i.setGeometry(None)

In [6]:
def RefPull(image):
    f = AddFmask(image).select('fmask')
    clouds = f.gte(2).rename('clouds')
    d = Dswe(image).select('dswe')
    dswe3 = d.eq(3).rename('dswe3').selfMask().updateMask(clouds.eq(0))
    dummy = image.select(['Blue'],['dswe1']).updateMask(clouds.eq(0)).updateMask(d.eq(1))
    cover = image.metadata('CLOUD_COVER')
    z = image.metadata('SUN_ELEVATION')
    a = image.metadata('SUN_AZIMUTH')
    
    pixOut = (image.addBands(d)
              .addBands(image.select(['Nir'],['NirSD']))
              .updateMask(d.eq(1))
              .updateMask(clouds.eq(0))
              .addBands(dswe3)
              .addBands(dummy)
              .addBands(clouds)
              .addBands(cover)
              .addBands(z)
              .addBands(a))
    
    ## Collect median reflectance and occurance values
    ## Make a cloud score, and get the water pixel count
    lsout = pixOut.reduceRegions(reach_poly, ee.Reducer.mean(), 30)
    
    return lsout

In [7]:
def maximum_no_of_tasks(MaxNActive, waitingPeriod):
  ##maintain a maximum number of active tasks
  time.sleep(10)
  ## initialize submitting jobs
  ts = list(ee.batch.Task.list())

  NActive = 0
  for task in ts:
       if ('RUNNING' in str(task) or 'READY' in str(task)):
           NActive += 1
  ## wait if the number of current active tasks reach the maximum number
  ## defined in MaxNActive
  while (NActive >= MaxNActive):
      time.sleep(waitingPeriod) # if reach or over maximum no. of active tasks, wait for 2min and check again
      ts = list(ee.batch.Task.list())
      NActive = 0
      for task in ts:
        if ('RUNNING' in str(task) or 'READY' in str(task)):
          NActive += 1
  return()

In [8]:
def applyScaleFactors89(image):
  opticalBands = image.select('SR_B.').multiply(0.0000275).add(-0.2)
  thermalBand = image.select('ST_B10').multiply(0.00341802).add(149.0)
  return image.addBands(srcImg = opticalBands, overwrite = True)\
              .addBands(srcImg = thermalBand, overwrite = True)

def applyScaleFactors75(image):
  opticalBands = image.select('SR_B.').multiply(0.0000275).add(-0.2)
  thermalBand = image.select('ST_B6').multiply(0.00341802).add(149.0)
  return image.addBands(srcImg = opticalBands, overwrite = True)\
              .addBands(srcImg = thermalBand, overwrite = True)

In [9]:
#### LOAD IN IN_SITU POINTS
points = ee.FeatureCollection("projects/ee-prumpunwath/assets/estuary_points")

wrs = ee.FeatureCollection("users/johngardner87/WRS2_descending")\
    .filterBounds(points)\
    .filterMetadata('MODE', 'equals', 'D')

In [10]:
# add dummy bands to fill in bands that landsat 8 has that 5 and 7 do not
dummyBands = ee.Image(-99).rename('Null_CS')\
    .addBands(ee.Image(-99).rename('Null_TIR2'))

# function to add dummy bands
def addDummy(i):
    return i.addBands(dummyBands)

l9 = ee.ImageCollection('LANDSAT/LC09/C02/T1_L2')
l8 = ee.ImageCollection("LANDSAT/LC08/C02/T1_L2")
l7 = ee.ImageCollection('LANDSAT/LE07/C02/T1_L2').map(addDummy)
l5 = ee.ImageCollection('LANDSAT/LT05/C02/T1_L2').map(addDummy)

bn98 = ['SR_B1','SR_B2','SR_B3', 'SR_B4', 'SR_B5','SR_B6','SR_B7', 'ST_B10','QA_PIXEL', 'ST_CDIST', 'ST_DRAD', 'ST_EMIS', 'ST_EMSD' ,'ST_QA',	'ST_TRAD', 'ST_URAD']
bn75 = ['Null_CS','SR_B1','SR_B2','SR_B3', 'SR_B4', 'SR_B5','SR_B7','ST_B6','QA_PIXEL', 'ST_CDIST', 'ST_DRAD',	'ST_EMIS',	'ST_EMSD' ,'ST_QA',	'ST_TRAD', 'ST_URAD']
bns = ['Aerosol','Blue', 'Green', 'Red', 'Nir', 'Swir1', 'Swir2', 'TIR1','pixel_qa','ST_CDIST', 'ST_DRAD', 'ST_EMIS', 'ST_EMSD'	,'ST_QA', 'ST_TRAD', 'ST_URAD']

ls9 = l9.map(applyScaleFactors89).select(bn98, bns)
ls8 = l8.map(applyScaleFactors89).select(bn98, bns)
ls7 = l7.map(applyScaleFactors75).select(bn75, bns)
ls5 = l5.map(applyScaleFactors75).select(bn75, bns)

ls = ls9.merge(ls8).merge(ls7).merge(ls5).filter(ee.Filter.lt('CLOUD_COVER', 50)).filterDate('1985-01-01', '2022-01-01')

pr = wrs.distinct('PR').sort('PR').aggregate_array('PR').getInfo() 
print(len(pr))

1229


In [12]:
df = pandas.DataFrame({'pr': pr}) 
df.to_csv('pr.csv', index=False) 

In [11]:
## loop through each landsast tile and apply function to make river masks and extract reflectance
for x in range(0,len(pr)):
    
## TEST for just 1 tile
#for x in range(0, 1):
    
    print(pr[x])
    
    tile = wrs.filterMetadata('PR', 'equals', pr[x])
    
    reach_poly = points.filterBounds(tile.geometry())
    
    stack = ls.filterBounds(tile.geometry().centroid())
    
    out = stack.map(RefPull).flatten()
    out = out.filter(ee.Filter.notNull(['Blue']))
  
# CHANGE folder output nanme to folder in your google drive to store output data
    dataOut = ee.batch.Export.table.toDrive(collection = out,\
                                            description = str(pr[x]),\
                                            folder = 'Estuary_collection2_Cloud_cover_50%',\
                                            fileFormat = 'csv')#,\
                                            #selectors  = ['Aerosol','Blue', 'Green', 'Red', 'Nir', 'Swir1', 'Swir2', 'TIR1', 'sd_NirSD', 'pixel_qa', 'clouds', 'dswe', 'hillShadow', 'pCount_dswe3','pCount_dswe1', 'pCount_shadow','system:index', 'CLOUD_COVER', 'SUN_ELEVATION', 'SUN_AZIMUTH',])    
  #Check how many existing tasks are running and take a break if it's >15  
    maximum_no_of_tasks(15, 60)
  # Send next task.
    dataOut.start()
    print('done')

# Make sure all Earth engine tasks are completed prior to moving on.  
maximum_no_of_tasks(1,300)
print('done')

001049
done
001050
done
001053
done
001075
done
001079
done
001081
done
001083
done
001084
done
001085
done
001086
done
001087
done
001088
done
001089
done
002053
done
002073
done
002075
done
003053
done
004047
done
004048
done
004053
done
005047
done
005048
done
005052
done
005053
done
006022
done
006023
done
006025
done
007022
done
007023
done
007025
done
007047
done
007052
done
007053
done
007054
done
008021
done
008022
done
008023
done
008025
done
008046
done
008047
done
008052
done
009013
done
009021
done
009022
done
009025
done
009026
done
009029
done
009047
done
009052
done
009053
done
009054
done
009066
done
010013
done
010025
done
010026
done
010028
done
010029
done
010053
done
010054
done
010055
done
010056
done
010057
done
010058
done
010059
done
010061
done
010062
done
011013
done
011025
done
011026
done
011029
done
011030
done
011031
done
011046
done
011048
done
011058
done
011059
done
011060
done
011061
done
011062
done
011063
done
012025
done
012026
done
012027
done
0120